# Lab-07-1 Tips

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as oiptim

In [2]:
torch.manual_seed(1)

### Training and Test Datasets

In [3]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

In [4]:
x_test = torch.FloatTensor( [[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

### Model

In [5]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,3)
    def forward(self, x):
        return self.linear(x)

In [7]:
model = SoftmaxClassifierModel()

In [21]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [23]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        prediction = model(x_train)
        cost = F.cross_entropy(prediction, y_train)
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Cost: {:6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [36]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)
    
    print('Accuracy: {}% Cost: {:.6f}'.format(
        correct_count / len(y_test) * 100, cost.item()
    ))

In [25]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 2.203667
Epoch    1/20 Cost: 1.199645
Epoch    2/20 Cost: 1.142985
Epoch    3/20 Cost: 1.117769
Epoch    4/20 Cost: 1.100901
Epoch    5/20 Cost: 1.089523
Epoch    6/20 Cost: 1.079872
Epoch    7/20 Cost: 1.071320
Epoch    8/20 Cost: 1.063325
Epoch    9/20 Cost: 1.055720
Epoch   10/20 Cost: 1.048378
Epoch   11/20 Cost: 1.041245
Epoch   12/20 Cost: 1.034285
Epoch   13/20 Cost: 1.027478
Epoch   14/20 Cost: 1.020813
Epoch   15/20 Cost: 1.014279
Epoch   16/20 Cost: 1.007872
Epoch   17/20 Cost: 1.001586
Epoch   18/20 Cost: 0.995419
Epoch   19/20 Cost: 0.989365


In [40]:
test(model, optimizer, x_test, y_test)

Accuracy: 0.0% Cost: 1.425844


### Data preprocessing (데이터 전처리)

In [42]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [43]:
mu = x_train.mean(0)
sigma = x_train.std(0)
norm_x_train = (x_train-mu) / sigma
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


In [44]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [45]:
model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [46]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [48]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost: 4.303410
Epoch    1/20 Cost: 2.891692
Epoch    2/20 Cost: 1.984457
Epoch    3/20 Cost: 1.400252
Epoch    4/20 Cost: 1.022918
Epoch    5/20 Cost: 0.778069
Epoch    6/20 Cost: 0.618177
Epoch    7/20 Cost: 0.512766
Epoch    8/20 Cost: 0.442335
Epoch    9/20 Cost: 0.394408
Epoch   10/20 Cost: 0.361000
Epoch   11/20 Cost: 0.336978
Epoch   12/20 Cost: 0.319069
Epoch   13/20 Cost: 0.305167
Epoch   14/20 Cost: 0.293913
Epoch   15/20 Cost: 0.284454
Epoch   16/20 Cost: 0.276220
Epoch   17/20 Cost: 0.268857
Epoch   18/20 Cost: 0.262139
Epoch   19/20 Cost: 0.255896


### Overfitting - Regularization

In [49]:
def train_with_regularization(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # l2 norm 계산
        l2_reg = 0
        for param in model.parameters():
            l2_reg += torch.norm(param)
            
        cost += l2_reg

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch+1, nb_epochs, cost.item()
        ))

In [50]:
model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [51]:
train_with_regularization(model, optimizer, norm_x_train, y_train)

Epoch    1/20 Cost: 29579.900391
Epoch    2/20 Cost: 18861.984375
Epoch    3/20 Cost: 12099.429688
Epoch    4/20 Cost: 7798.902344
Epoch    5/20 Cost: 5054.564453
Epoch    6/20 Cost: 3300.511963
Epoch    7/20 Cost: 2178.589355
Epoch    8/20 Cost: 1460.750610
Epoch    9/20 Cost: 1001.384277
Epoch   10/20 Cost: 707.401306
Epoch   11/20 Cost: 519.250854
Epoch   12/20 Cost: 398.830597
Epoch   13/20 Cost: 321.757019
Epoch   14/20 Cost: 272.425354
Epoch   15/20 Cost: 240.848572
Epoch   16/20 Cost: 220.635223
Epoch   17/20 Cost: 207.694687
Epoch   18/20 Cost: 199.408844
Epoch   19/20 Cost: 194.102203
Epoch   20/20 Cost: 190.702499
